# RCNN + PC model
Scripts for setting up our RCNN + PC model using tfomics (https://github.com/p-koo/tfomics)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
import sys
import h5py
import conutils
from sklearn.decomposition import PCA

from __future__ import print_function 
import os, sys
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf

sys.path.append('../Tensor/tfomics')
from tfomics import neuralnetwork as nn
from tfomics import utils, learn

# import models
from model_zoo import fourthplace_connectomics_model
from model_zoo import simple_connectomics_model, simple_connectomics_model2
from model_zoo import residual_connectomics_model, residual_connectomics_model2,residual_connectomics_model4

%matplotlib inline
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Load data -- from https://www.kaggle.com/c/connectomics/data
#
filename = '../Tensor/kaggle_connect_data/normal_dataset.hdf5'
group_name = ['normal_data']
dataset = h5py.File(filename,'r')
%time F_1 = np.array(dataset['/'+group_name[0]+'/F_1'])
scores_1 = np.array(dataset['/'+group_name[0]+'/scores_1'])
F_2 = np.array(dataset['/'+group_name[0]+'/F_2'])
scores_2 = np.array(dataset['/'+group_name[0]+'/scores_2'])
F_3 = np.array(dataset['/'+group_name[0]+'/F_3'])
scores_3 = np.array(dataset['/'+group_name[0]+'/scores_3'])
F_4 = np.array(dataset['/'+group_name[0]+'/F_4'])
scores_4 = np.array(dataset['/'+group_name[0]+'/scores_4'])

Wall time: 553 ms


In [3]:
# Load network positions for removing light scattering effects
#
pos = 'D:/Dropbox/Tensor/kaggle_connect_data/normal-1/networkPositions_normal-1.txt'
pos_1 = np.loadtxt(pos,delimiter=',')
F_1ls = conutils.unscatter(F_1.T,pos_1)

pos = 'D:/Dropbox/Tensor/kaggle_connect_data/normal-2/networkPositions_normal-2.txt'
pos_2 = np.loadtxt(pos,delimiter=',')
F_2ls = conutils.unscatter(F_2.T,pos_2)

pos = 'D:/Dropbox/Tensor/kaggle_connect_data/normal-3/networkPositions_normal-3.txt'
pos_3 = np.loadtxt(pos,delimiter=',')
F_3ls = conutils.unscatter(F_3.T,pos_3)

pos = 'D:/Dropbox/Tensor/kaggle_connect_data/normal-4/networkPositions_normal-4.txt'
pos_4 = np.loadtxt(pos,delimiter=',')
F_4ls = conutils.unscatter(F_4.T,pos_4)

In [5]:
# Calculate partial correlation metrics for all datasets
#
pred_out_1 = conutils.get_partial_corr_scores(F_1ls)
pred_out_2 = conutils.get_partial_corr_scores(F_2ls)
pred_out_3 = conutils.get_partial_corr_scores(F_3ls)
pred_out_4 = conutils.get_partial_corr_scores(F_4ls)

In [15]:
# Downsample data
#
ds_1 = conutils.roma_ds(F_1ls)
ds_2 = conutils.roma_ds(F_2ls)
ds_3 = conutils.roma_ds(F_3ls)
ds_valid = conutils.roma_ds(F_4ls)

NameError: name 'F_1ls' is not defined

In [9]:
# Z-score data
#
vs_1 = conutils.standardize_rows(ds_1)
vs_2 = conutils.standardize_rows(ds_2)
vs_3 = conutils.standardize_rows(ds_3)
vs_valid = conutils.standardize_rows(ds_valid)

In [11]:
# Standardize partial correlation coefficients
#
p1 = pred_out_1.copy()
p2 = pred_out_2.copy()
p3 = pred_out_3.copy()
p4 = pred_out_4.copy()

p1[p1==0] = np.min(p1[p1!=0])
p2[p2==0] = np.min(p2[p2!=0])
p3[p3==0] = np.min(p3[p3!=0])
p4[p4==0] = np.min(p4[p4!=0])

p1 = p1 - np.mean(p1)
p1 = p1/np.std(p1)

p2 = p2 - np.mean(p2)
p2 = p2/np.std(p2)

p3 = p3 - np.mean(p3)
p3 = p3/np.std(p3)

p4 = p4 - np.mean(p4)
p4 = p4/np.std(p4)

In [13]:
# Create data structure for model input
#
dtf, ltf = conutils.pairwise_prep_tuple_partialcorr((vs_1,vs_2,vs_3), 
                                                    (scores_1,scores_2,scores_3), 
                                                    (p1, p2, p3),
                                                   represent=50)

D:\Dropbox\TFconnect\conutils\utils.py:586: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  fluor_tf = np.empty((num_images, 4, num_samples, 1),dtype='float32')
D:\Dropbox\TFconnect\conutils\utils.py:587: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  label_tf = np.zeros((num_images,2),dtype='float32')
D:\Dropbox\TFconnect\conutils\utils.py:618: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  noncons_samp = (np.random.choice(noncons[0],(100-represent)*num_images/100,replace=False),
D:\Dropbox\TFconnect\conutils\utils.py:619: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  np.random.choice(noncons[1],(100-represent)*num_images/100,replace=False))


target size of processed traces: 1147744. count var: 1147744


In [15]:
# Separate data into training and cross-validation sets
#
inds = np.random.choice(dtf.shape[0],replace=False,size=dtf.shape[0])
dtf = dtf[inds,:,:,:]
ltf = ltf[inds]

crossval = dtf.shape[0]//4
dtf_crossval = dtf[:crossval,:,:,:]
ltf_crossval = ltf[:crossval,:]
dtf = dtf[crossval:,:,:,:]
ltf = ltf[crossval:,:]

In [3]:
X_train = dtf
y_train = ltf
X_valid = dtf_crossval
y_valid = ltf_crossval

In [4]:
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = residual_connectomics_model4.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, placeholders)
nnmodel.inspect_layers()

data_path = './'

# set output file paths
results_path = utils.make_directory(data_path, 'results')
output_name = 'dataset1_residual4'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 4, 330, 1)
layer2: conv1
(?, 3, 326, 32)
layer3: conv1_batch
(?, 3, 326, 32)
layer4: conv1_active
(?, 3, 326, 32)
layer5: conv1_dropout
(?, 3, 326, 32)
layer6: resid1_1resid
(?, 3, 326, 32)
layer7: resid1_1resid_norm
(?, 3, 326, 32)
layer8: resid1_1resid_active
(?, 3, 326, 32)
layer9: resid1_2resid
(?, 3, 326, 32)
layer10: resid1_2resid_norm
(?, 3, 326, 32)
layer11: resid1_resid_sum
(?, 3, 326, 32)
layer12: resid1_resid
(?, 3, 326, 32)
layer13: resid1_batch
(?, 3, 326, 32)
layer14: resid1_dropout
(?, 3, 326, 32)
layer15: conv2
(?, 1, 322, 64)
layer16: conv2_batch
(?, 1, 322, 64)
layer17: conv2_active
(?, 1, 322, 64)
layer18: conv2_dropout
(?, 1, 322, 64)
layer19: resid2_1resid
(?, 1, 322, 64)
layer20: resid2_1resid_norm
(?, 1, 322, 64)
layer21: resid2_1resid_active
(?, 1, 322, 64)
layer22: resid

In [8]:
# Train model
#
train = {'inputs': X_train, 'targets': y_train, 'keep_prob_conv': 0.8, 'keep_prob_dense': 0.5, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=100, num_epochs=200, 
                      patience=20, verbose=2, shuffle=True)

Epoch 1 out of 100 
total time: 434.15183210372925
num batches: 8609
  valid loss:		0.31012
  valid accuracy:	0.87814+/-0.00000
  valid auc-roc:	0.94269+/-0.00000
  valid auc-pr:		0.93975+/-0.00176
  saving model to:  D:/TFconnect/results\tfomics\trackall/fixed\testbad_partialcorr_2.ckpt
INFO:tensorflow:D:/TFconnect/results\tfomics\trackall/fixed\testbad_partialcorr_2.ckpt is not in all_model_checkpoint_paths. Manually adding it.


ValueError: setting an array element with a sequence.

In [14]:
val_dat = vs_valid
val_lbl = scores_4

In [21]:
# Evaluate model on validation data
#
pred_lbl =  conutils.valid_eval_tfomics_partialcorr(nntrainer,val_dat,p4)

C:\Users\Tim\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:225: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)


X
XX
XXX
XXXX
XXXXX
XXXXXX
XXXXXXX
XXXXXXXX
XXXXXXXXX
Wall time: 30min 44s


In [ ]:
# Get ROC-AUC metric
#
fpr, tpr, thresholds = roc_curve(np.reshape(val_lbl,(1e6,)), pred_lbl)
auc(fpr, tpr)

In [ ]:
def submission(pred_lbl_valid, pred_lbl_test):
    """
    Takes predicted labels / confidences for connections (number between 1 and 0) for both the kaggle
        validation dataset and test dataset. Outputs a competition-ready .csv file for submitting to
        leaderboard
    
    inputs---
        pred_lbl_valid: for validation set, 1-D numpy array with (num neurons)*(num neurons) 
            connection predictions. This is the predicted connectivity matrix stitched together
            row-wise (i.e. for 1000 neurons, first 1000 entries are connections from neuron 1 to
            neuron x)
        pred_lbl_test: same as above, for the test data
    """
    filename = './submission_pcorr_dataset3.csv'
    f = open(filename, 'w')
    
    # write header
    f.write('NET_neuronI_neuronJ,Strength\n')
    
    Nn = np.sqrt(len(pred_lbl_valid)).astype('int')

    cnt = 0
    for i in range(Nn):
        for j in range(Nn):
            f.write('valid_{}_{},{}\n'.format(i+1,j+1,pred_lbl_valid[cnt]))
            cnt += 1
    
    Nn = np.sqrt(len(pred_lbl_test)).astype('int')
    cnt = 0
    for i in range(Nn):
        for j in range(Nn):
            f.write('test_{}_{},{}\n'.format(i+1,j+1,pred_lbl_test[cnt]))
            cnt += 1
            
    f.close()

In [ ]:
submission(pred_lbl_val,pred_lbl_test)

In [ ]:
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob_conv': 1.0, 'keep_prob_dense': 1.0, 'is_training': False}
nntrainer.test_model(valid)

In [ ]:
f.close()

In [ ]:
from six.moves import cPickle

savefile = 'D:/Dropbox/TFconnect/results/tfomics/pickles/dataset3_partialcorr_endpoint_lowcc.pickle'
f = open(savefile, 'wb')
cPickle.dump(pred_lbl, f)
f.close()

In [31]:
from six.moves import cPickle

savefile = 'D:/Dropbox/TFconnect/results/tfomics/dataset7_residual4_partialcorr_crossval.pickle'
f = open(savefile, 'wb')
cPickle.dump(pred_lbl, f)
f.close()

In [32]:
savefile = 'D:/Dropbox/TFconnect/results/tfomics/dataset7_residual4_partialcorr_validation.pickle'
f = open(savefile, 'wb')
cPickle.dump(pred_lbl_val, f)
f.close()

savefile = 'D:/Dropbox/TFconnect/results/tfomics/dataset7_residual4_partialcorr_test.pickle'
f = open(savefile, 'wb')
cPickle.dump(pred_lbl_test, f)
f.close()

Let's look for possible common input motif problems and compare their partial correlations to real ones

In [ ]:
conn_mat_true = scores_1
TP = np.where(conn_mat_true)
TN = np.where(conn_mat_true==0)

In [ ]:
cnt = 0
poss_commons = []
poss_commons_size = []

for i in range(len(TN[0])):
    # Find false directed connection A->B 
    A = TN[0][i]
    B = TN[1][i]
    
    # Are the real connections C->A and C->B?
    check = np.intersect1d(TP[0][TP[1]==A],TP[0][TP[1]==B]).size
    if check != 0 :
        cnt += 1
        poss_commons.append((A,B))
        poss_commons_size.append(check)

In [ ]:
pcT = np.zeros((len(TP[0]),))
pcFP = np.zeros((len(poss_commons),))

In [ ]:
for i in range(len(pcT)):
    pcT[i] = pred_out[TP[0][i],TP[1][i]]

for i in range(len(pcFP)):
    pcFP[i] = pred_out[poss_commons[i][0],poss_commons[i][1]]

In [ ]:
plt.figure(figsize=(20,20))
plt.hist(pcFP,100,normed=1,range=[0.9,1])
plt.hist(pcT,100,normed=1,range=[0.9,1])

plt.show()

In [33]:
# Remember need to do 'dataset4_residual5_partialcorr_1d_conv_endpoint' separately because of diff. net structure

results_path = './results/tfomics/'

val_dat = vs_H

files = ['dataset1_partialcorr_endpoint_continued',
        'dataset2_1d_version_residual4_partialcorr_best',
        'dataset3_1d_version_residual4_partialcorr_best',
        'dataset5_residual4_partialcorr_2000batch_49',
        'dataset1_residual4_lowrep_partialcorr_best',
        'dataset6_residual4_derivnpdiffsignals_partialcorr_best',
        'dataset7_residual4_partialcorr_201']

for fn in files:
    nntrainer.set_best_parameters(os.path.join(results_path,fn+'.ckpt'))
    pred_lbl =  conutils.valid_eval_tfomics_partialcorr(nntrainer,val_dat,pH)
    

    savefile = os.path.join(results_path, fn+'_highcon.pickle')
    f = open(savefile, 'wb')
    cPickle.dump(pred_lbl, f)
    f.close()

val_dat = vs_L

for fn in files:
    nntrainer.set_best_parameters(os.path.join(results_path,fn+'.ckpt'))
    pred_lbl =  conutils.valid_eval_tfomics_partialcorr(nntrainer,val_dat,pL)
    

    savefile = os.path.join(results_path, fn+'_lowcon.pickle')
    f = open(savefile, 'wb')
    cPickle.dump(pred_lbl, f)
    f.close()
    

NameError: name 'vs_H' is not defined